# Time Series as Features

## Cycles
Cycles are patterns of growth and decay in a time series associated with how the value in a series at one time depends on values at previous times, but not necessarily on the time step itself.

Cyclic behavior vs Seasonality

* **Cyclic Behavior**:
  1. Cyclic behavior refers to repetitive patterns or fluctuations in data that occur over longer time intervals than seasonality.
  2. Cycles are often irregular in duration and amplitude, and they do not have fixed or predictable periods.
  3. Cycles typically result from economic, business, or environmental factors that operate over longer time horizons, such as business cycles, economic expansions and contractions, or long-term climate cycles.
  4. Unlike seasonality, cyclic behavior does not follow fixed calendar-based patterns and may not exhibit consistent periodicity over time.
* **Seasonality**:
  1. Seasonality refers to regular, predictable patterns or fluctuations in data that occur at fixed intervals or periods.
  2. Seasonal patterns repeat with a consistent frequency or period, such as daily, weekly, monthly, or yearly.
  3. Seasonality is often influenced by calendar effects, weather patterns, holidays, or other recurring events that occur on a regular basis.
  4. Seasonal patterns are typically more uniform and consistent compared to cyclic behavior, with clear peaks and troughs occurring at specific times of the year or other intervals.
  4. Seasonality can be decomposed into distinct seasonal components, such as daily, weekly, or monthly effects, which can be modeled and analyzed separately.

## Lagged Series and Lag Plots